<a id="top"></a>
# Tutorial Title

***

## Learning Goals
Write three to five learning goals. A learning goal should describe what a reader should know or be able to do by the end of the tutorial that they didn't know or couldn't do before.

```
By the end of this tutorial, you will:

- Use astroquery to access the HST and Gaia archives
- Understand the drizzle technique
- Run a source extractor on HST images.
- Cross match the HST and Gaia sources.
- Create your view own previews of targets for future proposals, including for JWST!

```

## Introduction
This tutorial teaches us to align images between HST observations using `drizzlepac` and `tweakreg`. We learn the subtleties of combining images from different filters in a single visit, and combining multiple visits. Image aligning and stacking is crucial to make the most of the extensive Hubble archives, and are a crucial tool in preparing proposals for future observations. 

Before starting, [set up AstroConda](https://astroconda.readthedocs.io/en/latest/installation.html). 

## Imports
Describe the libraries we're using here. If there's something unusual, explain what the library is, and why we need it.
- *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files
- *astropy.table Table* for creating tidy tables of the data
- *matplotlib.pyplot* for plotting data
- *astroquery* for pulling data from the Mikulski Archive at Space Telescope (MAST)
- *stwcs* for tracking the World Coordinate System (WCS) data in the image files
- *subprocess* to parallelize tasks
- *shutil*, *glob*, *os* for accessing and manipulating files in the local system 

If any of these modules are not currently installed, get them in your command line:

`conda install -u stwcs numpy matplotlib astropy astroquery shutil subprocess glob os` or 

`pip3 install -u stwcs numpy matplotlib astropy astroquery shutil subprocess glob os`


In [ ]:
%matplotlib inline
import numpy as np
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
from astroquery.mast import Mast
from astroquery.mast import Observations
import shutil, stwcs, subprocess, glob, os 

## Main Content
### Accessing Existing Images with astroquery.Observations

Let's start by retrieving images of our object of interest from the HST Archive. For this project, we will consider the Orion Nebula. Astroquery.Observations allows you to query targets by name, and it is helpful to additionally specify a search radius. We download the products and store them locally. 

More famous objects have been observed by Hubble many times, so the download may be prohibitively large. You may want to select only images in a certain band, or apply filters on any of the [fields listed here.]( https://mast.stsci.edu/api/v0/_c_a_o_mfields.html)

In [ ]:
obs_table = Observations.query_criteria(target_name=["Orion","OrionNebula"],intentType='science',dataproduct_type='image')
obs_table

It looks like Hubble has observed the Orion Nebula on 44 separate occasions. Let us download all of these. 

In [40]:
download_tab = Observations.download_products(obs_table['obsid'])
download_tab

Local Path,Status,Message,URL
str102,str8,object,object
./mastDownload/HST/j8up02usq/j8up02usq_flt_hlet.fits,COMPLETE,None,None
./mastDownload/HST/j8up02usq/j8up02usj_jif.fits,COMPLETE,None,None
./mastDownload/HST/j8up02usq/j8up02usj_jit.fits,COMPLETE,None,None
./mastDownload/HST/j8up02usq/j8up02usq_spt.fits,COMPLETE,None,None
./mastDownload/HST/j8up02usq/j8up02usq_trl.fits,COMPLETE,None,None
./mastDownload/HST/j8up02usq/j8up02usq_drz.jpg,COMPLETE,None,None
./mastDownload/HST/j8up02usq/j8up02usq_raw.jpg,COMPLETE,None,None
./mastDownload/HST/j8up02usq/j8up02usq_flt.jpg,COMPLETE,None,None
./mastDownload/HST/j8up02usq/j8up02usq_raw.fits,COMPLETE,None,None


In [47]:
!mkdir('OrionNebulaHST')
!cd OrionNebulaHST
!mkdir('FITS')
!mkdir('images')
for file in download_tab['Local Path']:
    os.rename(file, os.path.basename(file))
    
shutil.rmtree('mastDownload')
!mv *fits FITS/
!mv *jpg images/

FITS
hst_10057_02_acs_wfc_fr388n_j8up02uv_trl.txt
hst_10057_02_acs_wfc_fr388n_j8up02uw_trl.txt
hst_10057_02_acs_wfc_fr459m_j8up02v5_trl.txt
hst_10057_02_acs_wfc_fr459m_j8up02v7_trl.txt
hst_10057_02_acs_wfc_fr459m_j8up02v9_trl.txt
hst_10057_02_acs_wfc_fr505n_j8up02uy_trl.txt
hst_10057_02_acs_wfc_fr505n_j8up02v0_trl.txt
hst_10057_02_acs_wfc_fr656n_j8up02us_trl.txt
hst_10057_02_acs_wfc_fr656n_j8up02ut_trl.txt
hst_10057_02_acs_wfc_fr656n_j8up02uu_trl.txt
hst_10057_02_acs_wfc_fr914m_j8up02vb_trl.txt
hst_10057_02_acs_wfc_fr914m_j8up02vc_trl.txt
hst_10057_02_acs_wfc_fr914m_j8up02vd_trl.txt
images
previewing_transient_candidates.ipynb


At this point, all the FITS files live in the FITS directory, while pre-processed images are in images/. Take a moment to look through the images in your preferred image viewer. Next, we will load the FITS files and manipulate them.

## Assignment 1: 
Pull all available observations for the galaxy cluster Abell 2744, also known as Pandora's Cluster. Try your favourite object. If you query only by target name, do you get too many results? What filters might help you narrow down to the data most useful to your proposal? Besides images, what data types might serve you? 

In [ ]:
obs_table = Observations.query_object( `Fill out your query here` )


### Inspecting the observations

You will notice that the number of downloaded files far exceeds the number of observations collected in `obs_table`. To understand them, let us look into the one of the accompanying `.txt` files.

In [56]:
with open('hst_10057_02_acs_wfc_fr388n_j8up02uv_trl.txt') as f:
    a = f.readlines()
    for line in a[:10]:
        print(line.split('\n')[0])

2021330044237 INFO src=drizzlepac.hapsequencer- Run start time: 2021-11-26 04:42:37.777383
2021330044237 INFO src=drizzlepac.hapsequencer- Parse the poller and determine what exposures need to be combined into separate products.

2021330044237 INFO src=drizzlepac.haputils.poller_utils- Input image j8up02usq_flt.fits found in current working directory.
2021330044237 INFO src=drizzlepac.haputils.poller_utils- Input image j8up02utq_flt.fits found in current working directory.
2021330044237 INFO src=drizzlepac.haputils.poller_utils- Input image j8up02uuq_flt.fits found in current working directory.
2021330044237 INFO src=drizzlepac.haputils.poller_utils- Input image j8up02uvq_flt.fits found in current working directory.
2021330044237 INFO src=drizzlepac.haputils.poller_utils- Input image j8up02uwq_flt.fits found in current working directory.
2021330044237 INFO src=drizzlepac.haputils.poller_utils- Input image j8up02uyq_flt.fits found in current working directory.
2021330044237 INFO src=dri

This tells us that several files, j...fits, were collected at each observation and combined into one image per filter using `drizzlepac`. We can further use `drizzlepac` to combine all of these into a single composite image, deeper than any individual one.

The Drizzle technique is a way of orienting all the images into exactly the same reference frame. This is crucial since the telescope shifts its pointing direction slightly all the time; in addition, different instruments on the telescope point in different directions. To align the images, we first need to find all the point sources in them, and then feed in the point source lists, along with the WCS coordinate information, to `drizzlepac`.

### File Information

Each FITS file uses the Multi-Extension FITS (MEF) format, meaning that multiple each file contains multiple objects. The first object details how the file was processed, the second is the image, and subsequent files, if any, contain spectra and/or other data products. Inspecting the FITS files, we see that those with `drz` in the filename have already had AstroDrizzle run on them.

In [ ]:
h = fits.open('FITS/j8up02uvq_drz.fits')[0].header
h

If you would like to learn to drizzle-align images yourself, use use [this tutorial](http://localhost:8889/notebooks/notebooks/DrizzlePac/Initialization/Initialization.ipynb) to set up AstroDrizzle and [this one](http://localhost:8889/notebooks/notebooks/DrizzlePac/drizzle_wfpc2/drizzle_wfpc2.ipynb) to create your very own composite image.  

### Assignment 2
Inspect the header file. What else can you learn about the observation and its processing from this? How many files were combined? Were these in the same, or different filters? What inputs were provided to AstroDrizzle?

### Stacking images with tweakreg


***

## Exercises
Exercises are optional, but encouraged. Exercises can be woven into the main content of your tutorial, or appear in their own section toward the end of the tutorial. Final exercises can be more challenging, similar to homework problems. They can be minimal or take as long as 30 minutes to an hour to complete. If you do have one or more exercises in your tutorial, be sure to leave a blank code cell underneath each to show the reader that they're meant to try out their new skill right there. You may also want to include a "solutions" notebook next to your main notebook for the reader to check their work after they have finished their attempt.

## Additional Resources

This section is optional. Try to weave resource links into the main content of your tutorial so that they are falling in line with the context of your writing. For resources that do not fit cleanly into your narrative, you may include an additional resources section at the end of your tutorial. Usually a list of links using Markdown bullet list plus link format is appropriate:

- [MAST API](https://mast.stsci.edu/api/v0/index.html)
- [Kepler Archive Page (MAST)](https://archive.stsci.edu/kepler/)
- [Kepler Archive Manual](https://archive.stsci.edu/kepler/manuals/archive_manual.pdf)
- [Exo.MAST website](https://exo.mast.stsci.edu/)

## About this Notebook

**Author:** Urmila Chadayammuri, Postdoctoral Fellow, Smithsonian Astrophysical Observatory

**Updated On:** 2022-07-24

## Citations
Provide your reader with guidelines on how to cite open source software and other resources in their own published work.

```
If you use `astropy` or `lightkurve` for published research, please cite the
authors. Follow these links for more information about citing `astropy` and
`lightkurve`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `lightkurve`](http://docs.lightkurve.org/about/citing.html)

```

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 